In [10]:
import numpy as np
import random

import matplotlib.pyplot as plt
from qiskit import quantum_info as qi
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.primitives import StatevectorSampler, PrimitiveJob,StatevectorEstimator
from qiskit.circuit.library import TwoLocal,EfficientSU2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit,transpile
from qiskit import quantum_info
from qiskit_aer import AerSimulator 

In [11]:
def SQ_DAMPER(n):
    alice_choice=["I", "X", "H", "XH"]
    choice_list=[]
    qc=QuantumCircuit(n)
    for i in range(0,n):
            choice=random.choices(alice_choice, k = 1)
            choice_list.extend(choice)
            if(choice[0]=="I"):
                    None
            elif(choice[0]=="X"):
                    qc.x(i)
            elif(choice[0]=="H"):
                    qc.h(i)
            else:
                    qc.x(i)
                    qc.h(i)
    #-------------------------------------------
    #Generating bob's secret message 
    bin=[0,1] 
    bob_msg=[]
    for i in range(n):
        choice=random.choices(bin, k = 1)
        bob_msg.append(choice[0])
    #print("Bob secret message for alice which he is going to encode in the 2n message photons according to the encoding scheme :") 
    #print(bob_msg)   

    #creating 2n random single photons used as decoy qubits in Z-basis
    decoy_msg=[]
    for i in range(2*n):
        choice=random.choices(bin, k = 1)
        decoy_msg.append(choice[0])
    #print("state of decoy photons created randomly:") 
    #print(decoy_msg)  
    
    # ------------------------------------------
    
    #Genreating circuit for STR-1
    qc_str1=QuantumCircuit(4*n,2*n)
    #message encoding
    x=0
    for i in range(0,2*n,2):
        if(bob_msg[x]==0):
            None
        else:
            qc_str1.x(i+1)  
        x=x+1     


    #decoy qubits 
    x=0
    for i in range(2*n,4*n,1):
        if(decoy_msg[x]==0):
            None
        else:
            qc_str1.x(i)  
        x=x+1     

    #print("Quantum circuit of STR-1") 

    #----------------------------------------------

    M=n+2 #Creating sequence of length  M (M>n) by bob 
    M_msg=[]
    for i in range(M):
        choice=random.choices(bin, k = 1)
        M_msg.append(choice[0])
    #print("state of M photons created randomly:") 
    #print(M_msg)

    #n photons already created by alice and sent to bob
    #print("State of n photons sent by alice to bob:")
    #print(choice_list)

    #Final STR-2 string which the comination of M and n photons sent by bob
    #print()
    str_2=[]
    str_2.extend(M_msg)
    str_2.extend(choice_list)
    #print("STR-2: ",str_2)

    #-----------------------------------------------

    str_3=[]
    l=[]
    chosen_qubits=[] #to store the index of the chosen qubit to form STR-3 FROM STR-2
    for i in range(0,2*n):
        l.append(i)
    for i in l:
       c=random.choices(l, k = 1)
       chosen_qubits.append(c[0])
       str_3.append(str_2[c[0]])


    #-----------------------------------------------
    x=0
    qc_str3=QuantumCircuit(2*n,2*n)
    for i in range(2*n):
        if(str_3[i]==1 or str_3[i]=="X"):
            qc_str3.x(i)
        elif(str_3[i]=="H"):
            qc_str3.h(i)    
        elif(str_3[i]=="XH"):
            qc_str3.x(i)
            qc_str3.h(i) 
        else:
            None       

    #-------------------------------------------------

    alice_meas_choice=[]
    basis_choice=["X","Z"]
    for i in range(len(str_3)):
        choice=random.choices(basis_choice, k = 1)
        alice_meas_choice.append(choice[0])
        if(choice[0]=="X"):
            qc_str3.h(i)
            qc_str3.measure(i,i) #measuring in X-BASIS
        else:
            qc_str3.measure(i,i) #measuring in Z-BASIS 

    #-------------------------------------------------

    simulator_aer=AerSimulator()

    qc_aer=transpile(qc_str3,backend=simulator_aer) #transpiling the circuit based on the backend we choose
    result=simulator_aer.run(qc_aer,shots=1).result() #runnung our circuit using the simulator

    counts=result.get_counts()#getting counts of our measurement results
    #print(counts)
    l=list(counts.keys())
    c=l[0]

    #--------------------------------------------------

    CTRL_q=[]
    SIFT_q=[]
    for i in range(len(str_3)):
        if(str_3[i]==0 or str_3[i]==1):
          SIFT_q.append(i)
        else:
          CTRL_q.append(i)  

    #--------------------------------------------------

    count=0
    for i in CTRL_q:
     if((str_3[i]=="I" or str_3[i]=="X")  and alice_meas_choice[i]=="Z"):
        count=count+1
        #print("Basis matched in the following qubit:",i)
        if(str_3[i]=="I" and c[len(c)-1-i]=='0'):
            
            print("No EVE detected")
         
          
        elif(str_3[i]=="X" and c[len(c)-1-i]=='1'):
            
            print("No EVE detected")
        
        else:
            print(str_3[i],c[len(c)-1-i])
            
            print("EVE detected") 

     elif((str_3[i]=="H" or str_3[i]=="XH")  and alice_meas_choice[i]=="X"):
        count=count+1
        print("Basis matched in the following qubit:",i)
        if(str_3[i]=="H" and c[len(c)-1-i]=='0'):
            
            print("No EVE detected")
         
          
        elif(str_3[i]=="XH" and c[len(c)-1-i]=='1'):
            
            print("No EVE detected")
        
        else:
            print(str_3[i],c[len(c)-1-i])
            print("EVE detected---------------------------------------")

    #---------------------------------------------------------


    for i in range(len(str_3)):
        if(alice_meas_choice[i]=='Z'):  #checking only qubits measured by alice in Z-basis 
           if(str_3[i]==0 or str_3[i]==1): #checking only qubits beloging to M-msg in STR-3 (SIFT Qubits) not the n-photons which was checked in security check of alice
        
            index=chosen_qubits[i]
            print("check was done in the following qubit",i)
            print("index of the qubit in the original M-photon message",index)
            
            k=int(c[len(c)-1-i])
            if(k==(M_msg[index])): #comparing the initial message M created by bob with measurement result
                print('NO EVE DETECTED \n')
            else:
                print("EVE DETECTED----------------------------------- \n")   

    #--------------------------------------------------------------------------                 

    #Recreating the circuit of str_1 as alice discards 2n qubits
    qc_str1=QuantumCircuit(2*n,2*n)
    #message encoding
    x=0
    for i in range(0,2*n,2):
        if(bob_msg[x]==0):
            None
        else:
            qc_str1.x(i+1)  
        x=x+1    
    for i in range(2*n):
        qc_str1.measure(i,i)   

    #--------------------------------------------------------------------------

    simulator_aer=AerSimulator()

    qc_aer=transpile(qc_str1,backend=simulator_aer) #transpiling the circuit based on the backend we choose
    result=simulator_aer.run(qc_aer,shots=1).result() #runnung our circuit using the simulator

    counts=result.get_counts()#getting counts of our measurement results
    #print(counts)
    l=list(counts.keys())
    c=l[0]    


    #---------------------------------------------------------------------------

    decoded_key=[]
    for i in range(len(c)-1,-1,-2):
        if(c[i]==c[i-1]):
            decoded_key.append(0)
        else:
            decoded_key.append(1)

    #---------------------------------------------------------------------------

    bin=[0,1] 
    alice_msg=[]
    for i in range(n):
        choice=random.choices(bin, k = 1)
        alice_msg.append(choice[0]) 

    #--------------------------------------------------------------------------

    def xor_combination(list1, list2):
    
        if len(list1) != len(list2):
            print("generate list of same length")
        
        
        xor_list = [a ^ b for a, b in zip(list1, list2)]
        
        return xor_list

    # Example usage
    Final_key=xor_combination(bob_msg,alice_msg) 

    return Final_key


               

           


                    




{'11110111101101001111': 1}
check was done in the following qubit 0
index of the qubit in the original M-photon message 8
NO EVE DETECTED 

check was done in the following qubit 2
index of the qubit in the original M-photon message 10
NO EVE DETECTED 

check was done in the following qubit 3
index of the qubit in the original M-photon message 10
NO EVE DETECTED 

check was done in the following qubit 9
index of the qubit in the original M-photon message 11
NO EVE DETECTED 

check was done in the following qubit 10
index of the qubit in the original M-photon message 1
NO EVE DETECTED 

check was done in the following qubit 12
index of the qubit in the original M-photon message 10
NO EVE DETECTED 

check was done in the following qubit 13
index of the qubit in the original M-photon message 8
NO EVE DETECTED 

check was done in the following qubit 14
index of the qubit in the original M-photon message 9
NO EVE DETECTED 

check was done in the following qubit 17
index of the qubit in the o